This section imports the necessary libraries for data manipulation (pandas), using transformer models (transformers), handling tensors (torch), and evaluation metrics like accuracy and confusion matrices (sklearn).

In [ ]:
!pip install transformers datasets torch huggingface_hub

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

Loads the CSV file Reviews.csv into a Pandas DataFrame.
Combines the Summary and Text columns into a new column called combined_text to create a single text input for analysis, replacing NaN values with empty strings. Taking account of NaN because when we combine them if one of the column is NaN then it will return NaN.

In [ ]:
file_path = 'Reviews.csv' #Amazon Fine Food Reviews
data = pd.read_csv(file_path)

data['combined_text'] = data['Summary'].fillna('') + ". " + data['Text'].fillna('')

Loads pre-trained models for sentiment analysis: RoBERTa, BERT, and ALBERT.
Tokenizers for each model are also initialized to convert text into input suitable for the models.

In [ ]:
roberta_model_name = "cardiffnlp/twitter-roberta-base-sentiment"
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_name)
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_model_name)

bert_model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModelForSequenceClassification.from_pretrained(bert_model_name)

albert_model_name = "textattack/albert-base-v2-imdb"
albert_tokenizer = AutoTokenizer.from_pretrained(albert_model_name)
albert_model = AutoModelForSequenceClassification.from_pretrained(albert_model_name)

Prints the sentiment label mappings (id2label) of each model to understand their outputs, such as "negative," "neutral," or "positive."

In [ ]:
print(roberta_model.config.id2label)
print(bert_model.config.id2label)
print(albert_model.config.id2label)

Processes text using the RoBERTa tokenizer and model.
Computes sentiment probabilities for "negative," "neutral," and "positive" sentiments using softmax.

In [ ]:
def analyze_sentiment_roberta(text):
    try:
        tokens = roberta_tokenizer(text[:512], return_tensors="pt", truncation=True, padding=True)
        outputs = roberta_model(**tokens)
        scores = torch.nn.functional.softmax(outputs.logits, dim=1).detach().numpy()[0]
        return {
            'neg': scores[0],
            'neu': scores[1],
            'pos': scores[2],
        }
    except Exception as e:
        print(f"Error processing text: {e}")
        return {'neg': 0, 'neu': 0, 'pos': 0}

In [ ]:
def analyze_sentiment_bert(text):
    try:
        tokens = bert_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        outputs = bert_model(**tokens)
        logits = outputs.logits.detach().numpy()[0]
        scores = torch.nn.functional.softmax(torch.tensor(logits), dim=0).numpy()
        return scores

    except Exception as e:
        print(f"Error processing text: {e}")
        return None

In [ ]:
def analyze_sentiment_albert(text):
    try:
        tokens = albert_tokenizer(text[:512], return_tensors="pt", truncation=True, padding=True)
        outputs = albert_model(**tokens)
        scores = torch.nn.functional.softmax(outputs.logits, dim=1).detach().numpy()[0]
        return {
            'neg': scores[0],
            'pos': scores[1]
        }
    except Exception as e:
        print(f"Error processing text: {e}")
        return {'neg': 0, 'pos': 0}

In [ ]:
roberta_sentiments = data['combined_text'].apply(analyze_sentiment_roberta)
data['roberta_negativity'] = roberta_sentiments.apply(lambda x: x['neg'])
data['roberta_neutrality'] = roberta_sentiments.apply(lambda x: x['neu'])
data['roberta_positivity'] = roberta_sentiments.apply(lambda x: x['pos'])

In [ ]:
scores = data["combined_text"].apply(analyze_sentiment_bert)
for i in range(bert_model.config.num_labels):
    data[f"label_{i+1}"] = scores.apply(lambda x: x[i])

In [ ]:
albert_sentiments = data['combined_text'].apply(analyze_sentiment_albert)
data['albert_negativity'] = albert_sentiments.apply(lambda x: x['neg'])
data['albert_positivity'] = albert_sentiments.apply(lambda x: x['pos'])

In [ ]:
def categorize_score(score):
    if score <= 2:
        return 'negative'
    elif score == 3:
        return 'neutral'
    else:
        return 'positive'
data['Score_Category'] = data['Score'].apply(categorize_score)

In [ ]:
def classify_sentiment(row):
    if row["roberta_negativity"] > row["roberta_neutrality"] and row["roberta_negativity"] > row["roberta_positivity"]:
        return "negative"
    elif row["roberta_neutrality"] > row["roberta_negativity"] and row["roberta_neutrality"] > row["roberta_positivity"]:
        return "neutral"
    else:
        return "positive"
data["Roberta_Category"] = data.apply(classify_sentiment, axis=1)

data[['Id', 'combined_text', 'Score_Category', 'Roberta_Category']]

In [ ]:
data["Sentiment_Match_Roberta"] = data["Score_Category"] == data["Roberta_Category"]
match_percentage = data["Sentiment_Match_Roberta"].mean() * 100
mismatched_rows = data[data["Sentiment_Match_Roberta"] == False]
print(f"Percentage of matching predictions: {match_percentage:.2f}%")
print("Mismatched predictions:")
print(mismatched_rows[["Score_Category", "Roberta_Category"]])

In [ ]:
conf_matrix = confusion_matrix(data['Score_Category'], data['Roberta_Category'], labels=['positive', 'neutral', 'negative'])
class_report = classification_report(data['Score_Category'], data['Roberta_Category'], target_names=['positive', 'neutral', 'negative'])
print("Confusion Matrix of Roberta:")
print(conf_matrix)
print("\nClassification Report of Roberta:")
print(class_report)

In [ ]:
#BERT
label_columns = ['label_1', 'label_2', 'label_3', 'label_4', 'label_5']
data['Predicted_Sentiment_BERT'] = data[label_columns].idxmax(axis=1).str.extract('(\d)').astype(int)
class_report = classification_report(data['Score'], data['Predicted_Score'])
accuracy = accuracy_score(data['Score'], data['Predicted_Score'])
conf_matrix = confusion_matrix(data['Score'], data['Predicted_Score'])
print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix of BERT:")
print(conf_matrix)
print("\nClassification Report of BERT:")
print(class_report)

In [ ]:
#ALBERT
def predict_sentiment(row):
    if row['albert_positivity'] > 0.5:
        return 'positive'
    elif row['albert_negativity'] > 0.5:
        return 'negative'
    else:
        return 'neutral'

data['Predicted_Sentiment_Albert'] = data.apply(predict_sentiment, axis=1)

conf_matrix = confusion_matrix(data['Score_Category'], data['Predicted_Sentiment_Albert'], labels=['positive', 'neutral', 'negative'])
class_report = classification_report(data['Score_Category'], data['Predicted_Sentiment_Albert'], target_names=['positive', 'neutral', 'negative'])

print("Confusion Matrix of Albert:")
print(conf_matrix)
print("\nClassification Report of Albert:")
print(class_report)

### Additional Analysis
Identifies specific reviews where predictions are interesting or potentially mismatched for further investigation.

In [ ]:
data.query('Score == 1').sort_values('roberta_positivity', ascending=False).iloc[3]["Text"]

In [ ]:
data.query('Score == 5').sort_values('roberta_negativity', ascending=False).iloc[1]["Text"]